In [7]:
import os, sys, getopt
from psutil import virtual_memory
sys.path.insert(0, "cdc/src/")
from NoteDeid import *
from NoteConceptParser import *
from Converter import *
from MLPipeline import *
from D2v import *
from RPDR import *
def evaluate(argv):    
    argv=argv.split()
    mem = virtual_memory().total / 1024.**3
    
    workdir = datadir = labelfile = deid = parser = parserdir = ctakessetup = utsname = utspw = \
    ctakesminmem = ctakesmaxmem = convert = model = feature = vec_represent = algorithm = \
    balanced_class = repeat = cv = rpdr = ''
    
    try:
        opts, args = getopt.getopt(argv, "w:d:l:i:p:q:s:n:o:e:g:c:m:f:v:a:b:r:k:x:h", \
        ["workdir=", "datadir=", "labelfile=", "deid=", "parser=", "parserdir=", "ctakessetup=", "utsname=", \
        "utspw=", "ctakesminmem=", "ctakesmaxmem=", "convert=", "model=", "feature=", "vec_represent=", \
        "algorithm=", "balancedclass=", "repeat=", "cv=", "rpdr=", "help"])
    except getopt.GetoptError:
        usage('pipeline.py -wd <workdir> -dd <datadir> -lf <labelfile> -d <T/F> -p <T/F> -pd <parserdir> -cset <T/F> -un <utsname> -upw <utspw> -mnm <ctakesminmem> -mxm <ctakesmaxmem> -c <convert> -m <T/F> -f <feature> -v <vec_represent> -a <algorithm> -bc <T/F> -r <repeat> -cv <cv> -h')
        sys.exit(2)
    
    for opt, arg in opts:
        if opt == ("-h", "--help"):
            usage('pipeline.py -wd <workdir> -dd <datadir> -lf <labelfile> -d <T/F> -p <T/F> -pd <parserdir> -cset <T/F> -un <utsname> -upw <utspw> -mnm <ctakesminmem> -mxm <ctakesmaxmem> -c <convert> -m <T/F> -f <feature> -v <vec_represent> -a <algorithm> -bc <T/F> -r <repeat> -cv <cv> -h')
            sys.exit()
        elif opt in ("-w", "--workdir"):
            workdir = arg
        elif opt in ("-d", "--datadir"):
            datadir = arg
        elif opt in ("-l", "--labelfile"):
            labelfile = arg
        elif opt in ("-i", "--deid"):
            deid = arg
        elif opt in ("-s", "--ctakessetup"):
            ctakessetup = arg
        elif opt in ("-n", "--utsname"):
            utsname = arg
        elif opt in ("-o", "--utspw"):
            utspw = arg
        elif opt in ("-e", "--ctakesminmem"):
            ctakesminmem = arg
        elif opt in ("-g", "--ctakesmaxmem"):
            ctakesmaxmem = arg
        elif opt in ("-m", "--model"):
            model = arg
        elif opt in ("-f", "--feature"):
            feature = arg
        elif opt in ("-v", "--vec_represent"):
            vec_represent = arg
        elif opt in ("-a", "--algorithm"):
            algorithm = arg
        elif opt in ("-b", "--balancedclass"):
            balanced_class = arg
        elif opt in ("-r", "--repeat"):
            repeat = int(arg)
        elif opt in ("-k", "--cv"):
            cv = int(arg)
        elif opt in ("-x", "--rpdr"):
            xx = arg
    
    print 'Working directory        :', workdir
    print 'Data directory           :', datadir
    print 'Label file location      :', labelfile
    print 'Running deidentification :', deid
    print 'Setting up cTAKES        :', ctakessetup
    print 'UTS username             :', utsname
    print 'UTS password             :', utspw
    print 'cTAKES minimal memory    :', ctakesminmem
    print 'cTAKES maximal memory    :', ctakesmaxmem
    print 'Data conversion from     :', convert
    print 'Modeling                 :', model
    print 'Selected feature         :', feature
    print 'Vector representation    :', vec_represent
    print 'Algorithm                :', algorithm
    print 'Class balancing          :', balanced_class
    print 'Repeat time              :', repeat
    print 'k-fold cross-validation  :', cv

    if os.path.exists(workdir) == False:
        os.system("mkdir " + workdir + "; cd " + workdir + "; mkdir data; mkdir model; mkdir result; cd ..")
        
    os.chdir(workdir)
    
    if datadir == '' and 'xx' not in locals():
        datadir = workdir + 'data/'
        print "No data provided, use sample data"
        os.system("wget https://github.com/ckbjimmy/ckbjimmy.github.io/raw/master/files/sample.tar.gz; \
        wget https://raw.githubusercontent.com/ckbjimmy/ckbjimmy.github.io/master/files/label.txt; \
        tar -xzf sample.tar.gz; mv sample/ data/xml/; mv label.txt data/label.txt; rm sample.tar.gz")
        labelfile = datadir + 'label.txt'
        
    if deid == "T":
        try: 
            RunDeid(folder=datadir, deidDir=workdir + 'deid-1.1')
        except:
            print "Downloading deid package from physionet"
            os.chdir(workdir)
            os.system("wget https://www.physionet.org/physiotools/sources/deid/deid-1.1.tar.gz; \
        tar -xzf deid-1.1.tar.gz; rm deid-1.1.tar.gz; cd deid-1.1; rm *.txt; cd ..")
            try:
                RunDeid(folder=datadir, deidDir=workdir + 'deid-1.1')
            except:
                print "No free text files for deidentification"
                sys.exit(2)

    if model == "T":
        if convert == "idash":
            MLPipeline(path=workdir, data=workdir + 'data/data.txt', label=workdir + 'data/label.txt', \
            vec=vec_represent, alg = algorithm, cwt=balanced_class, feature=feature, rep=repeat, k=cv)
        elif convert == "rpdr":
            rpdr_param = xx.split('+')
            RPDR(path=workdir, lab=rpdr_param[1], num=rpdr_param[2])
            MLPipeline(path=workdir, data=workdir + 'data/data.txt', label=workdir + 'data/label.txt', \
            vec=vec_represent, alg = algorithm, cwt=balanced_class, feature=feature, rep=repeat, k=cv)
        else:
            MLPipeline(path=workdir, data=workdir + 'data/data.txt', label=labelfile, \
            vec=vec_represent, alg = algorithm, cwt=balanced_class, feature=feature, rep=repeat, k=cv)
        
    print "Done!"

In [ ]:
project_dir="/Users/zebinkang/Github/scalable_text_analysis/"  #modify to your project directory
evaluate('-w'+project_dir+'test/ -d '+project_dir+'test/data/data.txt -l '+project_dir+'test/data/label.txt -m T -f sg -v freq '+
         '-a l1+l2+nb+svmlin+svmsgd+mlp_lbfgs+mlp_sgd+mlp_adam -b None -r 1 -k 5 | tee ../log_test.txt')

Working directory        : /Users/zebinkang/Github/scalable_text_analysis/test/
Data directory           : /Users/zebinkang/Github/scalable_text_analysis/test/data/data.txt
Label file location      : /Users/zebinkang/Github/scalable_text_analysis/test/data/label.txt
Running deidentification : 
Setting up cTAKES        : 
UTS username             : 
UTS password             : 
cTAKES minimal memory    : 
cTAKES maximal memory    : 
Data conversion from     : 
Modeling                 : T
Selected feature         : sg
Vector representation    : freq
Algorithm                : l1+l2+nb+svmlin+svmsgd+mlp_lbfgs+mlp_sgd+mlp_adam
Class balancing          : None
Repeat time              : 1
k-fold cross-validation  : 5
--- Data preprocessing ---
Label encoding
--- Vector representation ---
One-hot representation
Convert to sparse matrix
(431, 4531)
Feature selection (N)
Topic modeling (N)
--- Supervised learning with repeated cross-validation ---
input X, y, algorithm, rep, k
Model: sg | Logis

/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Model: sg | LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) | rep1 | cv4 | time: 2.0318031311 sec
(0.82352941176470584, 0.82839215686274503, 0.82352941176470584, 0.82400772430184188, 0.88937534098824422)
Model: sg | LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) | rep1 | cv5 | time: 2.02887010574 sec
(0.89411764705882357, 0.89694864612511671, 0.89411764705882357, 0.89284744667097615, 0.9315680417150406)
Model: sg | LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_j

Model: sg | CalibratedClassifierCV(base_estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=0, shuffle=True,
       tol=None, verbose=0, warm_start=False),
            cv=5, method='sigmoid') | rep1 | cv5 | time: 2.19399809837 sec
(0.88235294117647056, 0.89453620528671229, 0.88235294117647056, 0.88214752567693755, 0.92428967690979391)
Model: sg | MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(128, 64, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False

/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Model: sg | MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(128, 64, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False) | rep1 | cv1 | time: 29.0486571789 sec
(0.8089887640449438, 0.84046865345783495, 0.8089887640449438, 0.79406820267253453, 0.87179398581085799)
Model: sg | MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(128, 64, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1,

/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model: sg | MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(128, 64, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False) | rep1 | cv3 | time: 32.1805472374 sec
(0.84705882352941175, 0.80764318034906268, 0.84705882352941175, 0.82029411764705873, 0.90018276032975897)
Model: sg | MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(128, 64, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.